In [ ]:
# pyright: reportMissingImports=false
import sys
import os
import time
import logging
import datetime
from dotenv import load_dotenv
import pandas as pd
import numpy as np
import requests
import pandas_gbq
from dreams_core.googlecloud import GoogleCloud as dgc
from dreams_core import core as dc
import matplotlib.pyplot as plt

# load dotenv
load_dotenv()

# configure logger
logging.basicConfig(
    level=logging.INFO,
    format='[%(asctime)s] %(levelname)s [%(name)s.%(funcName)s:%(lineno)d] %(message)s',
    datefmt='%d/%b/%Y %H:%M:%S'
    )
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)

## import local files if necessary
# sys.path.append('../GitHub/core-functions/src/dreams_core')
# import googlecloud as dgc2
# importlib.reload(dgc2)

# sys.path.append('../GitHub/etl-pipelines/coin_wallet_metrics')
# import coin_wallet_met?rics as wm
# importlib.reload(wm)


# production-ready code

In [ ]:
query_sql = '''
    select * from analytics.bitcoin_prices
    '''
prices_df_full = dgc().run_sql(query_sql)
print(prices_df_full.info())
prices_df_full.head()

# code drafting workspace

In [ ]:
def classify_price_action(prices_df_full, cutoff_date, window, price_activity_threshold, period_min_duration):
    """
    Calculate the period state for a given price DataFrame.

    Parameters:
        prices_df_full (pd.DataFrame): The full DataFrame containing date and price data.
        cutoff_date (str): The cutoff date in 'YYYY-MM-DD' format to filter the DataFrame.
        window (int): The window size for calculating rolling mean and standard deviation.
        price_activity_threshold (float): The average daily return threshold required for classifying \
            a period as Bullish or Bearish.
        period_min_duration (int): the number of days of bullish or bearish behavior to escape \
            classification as "Consolidation"

    Returns:
        pd.DataFrame: A DataFrame with an additional 'period' column classifying each row as 'Bullish', 'Bearish', or 'Consolidation'.
    """
    # 1. Calculate percentage changes within cutoff date
    # --------------------------------------------------
    prices_df = prices_df_full[prices_df_full['date'] >= cutoff_date].copy()

    # Calculate percentage changes
    prices_df['date'] = pd.to_datetime(prices_df['date'])
    prices_df.sort_values('date', inplace=True)
    prices_df['daily_pct_change'] = prices_df['price'].pct_change()

    # Calculate rolling mean percentage change
    prices_df['rolling_mean_long'] = prices_df['daily_pct_change'].rolling(window=window).mean()



    # 2. Classify long term and short term periodic price action
    # ----------------------------------------------------------
    # Long term periods
    def classify_period(row):
        if row['rolling_mean_long'] >= price_activity_threshold:
            return 'Bullish'
        elif row['rolling_mean_long'] <= -price_activity_threshold:
            return 'Bearish'
        else:
            return 'Consolidation'

    prices_df['period_long'] = prices_df.apply(classify_period, axis=1)

    # Short term periods
    prices_df['rolling_mean_short'] = prices_df['daily_pct_change'].rolling(window=3).mean()

    def classify_period(row):
        if row['rolling_mean_short'] >= price_activity_threshold*2:
            return 'Bullish'
        elif row['rolling_mean_short'] <= -price_activity_threshold*2:
            return 'Bearish'
        else:
            return 'Consolidation'

    prices_df['period_short'] = prices_df.apply(classify_period, axis=1)

    # 3. Classify overall price action based on long and short term behavior
    # ----------------------------------------------------------------------
    def determine_period(row):
        if row['period_long'] == 'Bullish':
            return 'Bullish'
        elif row['period_long'] == 'Bearish':
            return 'Bearish'
        elif row['period_long'] == 'Consolidation' and row['period_short'] == 'Bullish':
            return 'Bullish'
        elif row['period_long'] == 'Consolidation' and row['period_short'] == 'Bearish':
            return 'Bearish'
        else:
            return 'Consolidation'

    prices_df['period'] = prices_df.apply(determine_period, axis=1)

    # 4. Apply period_min_duration
    # ----------------------------
    # Add a column to calculate how long many days a period has extended
    prices_df['period_count'] = prices_df.groupby((prices_df['period'] != prices_df['period'].shift()).cumsum()).cumcount() + 1

    # Calculate the total duration of a given period
    prices_df['period_duration'] = prices_df.groupby((prices_df['period'] != prices_df['period'].shift()).cumsum())['period_count'].transform('max')

    # Convert any periods below period_min_duration to Consolidation
    prices_df['period_adj'] = prices_df['period']
    prices_df.loc[prices_df['period_duration'] < period_min_duration, 'period_adj'] = 'Consolidation'

    return prices_df


def graph_price_action(prices_df):
    """
    Graph the price action of a given DataFrame with color-coded periods.

    Parameters:
    prices_df (pd.DataFrame): The DataFrame containing date, price, and period data.
                              Assumes 'date', 'price', and 'period' columns exist in the DataFrame.

    Returns:
    None
    """
    # Set colors for different periods
    colors = {'Bullish': 'green', 'Bearish': 'red', 'Consolidation': '#D3D3D3'}

    # Map the period to colors
    prices_df['color'] = prices_df['period_adj'].map(colors)

    # Create a figure and axis
    fig, ax = plt.subplots(figsize=(21, 10))

    # Plot the price data with color coding based on the period
    ax.plot(prices_df['date'], prices_df['price'], color='blue', label='_nolegend_')

    # Apply color to line segments based on the period
    for i in range(len(prices_df) - 1):
        ax.plot(prices_df['date'].iloc[i:i+2], prices_df['price'].iloc[i:i+2], color=prices_df['color'].iloc[i])

    # Set the y-axis to log scale
    ax.set_yscale('log')

    # Add labels and title
    ax.set_xlabel('Date')
    ax.set_ylabel('Price')
    ax.set_title('Bitcoin Price with Periods Highlighted')

    # Add a legend
    handles = [plt.Line2D([0], [0], color=color, label=label) for label, color in colors.items()]
    ax.legend(handles=handles)

    # Show the plot
    plt.show()



In [ ]:
# Define the rolling window period
window = 30

# Define daily average change thresholds
price_activity_threshold = 0.0035

# cutoff date
cutoff_date = '2014-01-01'

# minimum length of bullish or bearish period
period_min_duration = 30

prices_df = classify_price_action(prices_df_full, cutoff_date, window, price_activity_threshold, period_min_duration)
graph_price_action(prices_df)
# prices_df.tail(20)

In [ ]:
graph_price_action(prices_df)

In [ ]:
pd.set_option('display.max_rows', 100)


start_date = '2024-02-01'
end_date = '2024-04-01'

prices_df[(prices_df['date'] >= start_date) & (prices_df['date'] <= end_date)]

In [ ]:
pd.set_option('display.max_rows', 100)


start_date = '2023-10-01'
end_date = '2024-01-01'

filtered_df = prices_df[(prices_df['date'] >= start_date) & (prices_df['date'] <= end_date)]

filtered_df

In [ ]:
jupyter nbextension enable --py --sys-prefix qgrid
jupyter nbextension enable --py --sys-prefix widgetsnbextension


In [ ]:
import qgrid
import pandas as pd

# Example DataFrame
data = {
    'A': [1, 2, 3, 4],
    'B': [5, 6, 7, 8],
    'C': [9, 10, 11, 12]
}
df = pd.DataFrame(data)

# Display DataFrame using qgrid
qgrid_widget = qgrid.show_grid(df, show_toolbar=True)
qgrid_widget


In [ ]:
prices_df['period'] = prices_df.apply(
    lambda row: 'Consolidation' if row['period_long'] == 'Consolidation' and row['period_short'] == 'Consolidation' else (
        'Consolidation' if row['period_long'] != row['period_short'] else (
            'Bearish' if 'Bearish' in [row['period_long'], row['period_short']] else 'Bullish'
        )
    ), axis=1
)


prices_df['period'] = prices_df.apply(
    lambda row: 'Bearish' if 'Bearish' in [row['period_long'], row['period_short']] else (
        'Bullish' if 'Bullish' in [row['period_long'], row['period_short']]
        else 'Consolidation'
    ), axis=1
)

prices_df[30:70]

prices_df[30:70]

In [ ]:
# Add a column to calculate consecutive periods
prices_df['period_count'] = prices_df.groupby((prices_df['period'] != prices_df['period'].shift()).cumsum()).cumcount() + 1


# Add a column to calculate the duration of each period
prices_df['period_duration'] = prices_df.groupby((prices_df['period'] != prices_df['period'].shift()).cumsum())['period_count'].transform('max')


prices_df[30:70]